<a href="https://colab.research.google.com/github/alihaider-ml/Pyspark/blob/main/PySpark_Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Library

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, sum, round

## Start Spark Session

In [ ]:
spark = SparkSession.builder.appName("Lab3").getOrCreate()

## Load Dataset

In [ ]:
import pandas as pd

excel_file_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"

# Read the Excel file into a pandas DataFrame first
df_pandas = pd.read_excel(excel_file_path)


In [ ]:
df_pandas.to_csv("online_retail.csv", index=False)

In [ ]:
df = spark.read.csv("online_retail.csv", header=True, inferSchema=True)
# if we not use inferschema it will take all the data types as string

## **Explore Dataset**

In [ ]:
df.show(truncate = 0)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |22752

### Sort the data

In [ ]:
#order by is used to sort the data
df.orderBy('Quantity', ascending=False).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   581483|    23843|PAPER CRAFT , LIT...|   80995|2011-12-09 09:15:00|     2.08|   16446.0|United Kingdom|
|   541431|    23166|MEDIUM CERAMIC TO...|   74215|2011-01-18 10:01:00|     1.04|   12346.0|United Kingdom|
|   578841|    84826|ASSTD DESIGN 3D P...|   12540|2011-11-25 15:57:00|      0.0|   13256.0|United Kingdom|
|   542504|    37413|                NULL|    5568|2011-01-28 12:03:00|      0.0|      NULL|United Kingdom|
|   573008|    84077|WORLD WAR 2 GLIDE...|    4800|2011-10-27 12:26:00|     0.21|   12901.0|United Kingdom|
|   554868|    22197|SMALL POPCORN HOLDER|    4300|2011-05-27 10:52:00|     0.72|   13135.0|United Kingdom|
|   556231|   85123A|       

### Count Rows

In [ ]:
df.count()

541909

In [ ]:
# for any column
df.select('Quantity').count()

541909

### Unique values

In [ ]:
df.select('Country').distinct().show()

+------------------+
|           Country|
+------------------+
|            Sweden|
|         Singapore|
|           Germany|
|            France|
|            Greece|
|European Community|
|           Belgium|
|           Finland|
|             Malta|
|       Unspecified|
|             Italy|
|              EIRE|
|         Lithuania|
|            Norway|
|             Spain|
|           Denmark|
|         Hong Kong|
|           Iceland|
|            Israel|
|   Channel Islands|
+------------------+
only showing top 20 rows



### Unique count

In [ ]:
df.select('country').distinct().count()

38

In [ ]:
df.describe().show()

+-------+------------------+------------------+--------------------+------------------+-----------------+------------------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|        UnitPrice|        CustomerID|    Country|
+-------+------------------+------------------+--------------------+------------------+-----------------+------------------+-----------+
|  count|            541909|            541909|              540455|            541909|           541909|            406829|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0|  9.55224954743324| 4.61111362608298|15287.690570239585|       NULL|
| stddev|13428.417280803813|16799.737628427752|                NULL|218.08115785023477|96.75985306117967|1713.6003033215982|       NULL|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -80995|        -11062.06|           12346.0|  Australia|
|    max|           C581569|             

In [ ]:
for col in df.columns:
    print(f'{col} has {df.select(col).distinct().count()} values.')

InvoiceNo has 25900 values.
StockCode has 4070 values.
Description has 4224 values.
Quantity has 722 values.
InvoiceDate has 23260 values.
UnitPrice has 1630 values.
CustomerID has 4373 values.
Country has 38 values.


### **Add a new column**

#### **Column filled with a constant value**

In [ ]:
df.columns()

TypeError: 'list' object is not callable

In [ ]:
df.withColumn('Extra', lit('Ali')).show()
#lit is used to add any constant value in the new column in the dataset
#this data can't make changes to the our dataset permanently

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Extra|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|  Ali|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|  Ali|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   1785

In [ ]:
# for changing our dataset permanently
df = df.withColumn('Extra', lit('Ali'))
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Extra|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|  Ali|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|  Ali|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   1785

#### **Column Based on expression**

In [ ]:
from pyspark.sql.functions import col

df = df.withColumn("sales", col("Quantity") * col("UnitPrice"))
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Extra|             sales|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|  Ali|15.299999999999999|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|             20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|  Ali|              22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|             20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:0

In [ ]:
df = df.withColumn("Increament", col("Sales") + col('Quantity'))
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Extra|             sales|        Increament|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|  Ali|15.299999999999999|21.299999999999997|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|             20.34|             26.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|  Ali|              22.0|              30.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   1785

#### **Column with condition**

In [ ]:
df = df.withColumn('isBigOrder', when(col('Quantity')>5000, "Yes its big order").otherwise("Not a big order"))
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+------------------+---------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Extra|             sales|        Increament|     isBigOrder|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+------------------+---------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|  Ali|15.299999999999999|21.299999999999997|Not a big order|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|             20.34|             26.34|Not a big order|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|  Ali|              22.0|              30.0|Not a big

In [ ]:
df.orderBy('Quantity', ascending=False).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+------------------+-----------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Extra|             sales|        Increament|       isBigOrder|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+------------------+-----------------+
|   581483|    23843|PAPER CRAFT , LIT...|   80995|2011-12-09 09:15:00|     2.08|   16446.0|United Kingdom|  Ali|          168469.6|          249464.6|Yes its big order|
|   541431|    23166|MEDIUM CERAMIC TO...|   74215|2011-01-18 10:01:00|     1.04|   12346.0|United Kingdom|  Ali|           77183.6|          151398.6|Yes its big order|
|   578841|    84826|ASSTD DESIGN 3D P...|   12540|2011-11-25 15:57:00|      0.0|   13256.0|United Kingdom|  Ali|               0.0|           12540.0

In [ ]:
# more then one condition
df = df.withColumn("Order_type",when(col("Quantity")>5000, "Big Order").when(col("Quantity")>3000, "Medium").otherwise("small"))
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+------------------+---------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Extra|             sales|        Increament|     isBigOrder|Order_type|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+------------------+---------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|  Ali|15.299999999999999|21.299999999999997|Not a big order|     small|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|             20.34|             26.34|Not a big order|     small|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdo

In [ ]:
#task
df = df.withColumn("price_category", when(col("UnitPrice")>15000, "Expensive").when(col("UnitPrice")>5000, "Normal").otherwise("Not Expensive"))
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+------------------+---------------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Extra|             sales|        Increament|     isBigOrder|Order_type|price_category|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+------------------+------------------+---------------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|  Ali|15.299999999999999|21.299999999999997|Not a big order|     small| Not Expensive|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Ali|             20.34|             26.34|Not a big order|     small| Not Expensive|
|   536365|   84406B|CREAM CUP

In [ ]:
df.orderBy("UnitPrice", ascending=False).show()

+---------+---------+---------------+--------+-------------------+---------+----------+--------------+-----+---------+----------+---------------+----------+--------------+
|InvoiceNo|StockCode|    Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Extra|    sales|Increament|     isBigOrder|Order_type|price_category|
+---------+---------+---------------+--------+-------------------+---------+----------+--------------+-----+---------+----------+---------------+----------+--------------+
|  C556445|        M|         Manual|      -1|2011-06-10 15:31:00|  38970.0|   15098.0|United Kingdom|  Ali| -38970.0|  -38971.0|Not a big order|     small|     Expensive|
|  C580605|AMAZONFEE|     AMAZON FEE|      -1|2011-12-05 11:36:00| 17836.46|      NULL|United Kingdom|  Ali|-17836.46| -17837.46|Not a big order|     small|     Expensive|
|  C540117|AMAZONFEE|     AMAZON FEE|      -1|2011-01-05 09:55:00| 16888.02|      NULL|United Kingdom|  Ali|-16888.02| -16889.02|Not a big o

### **Drop Columns**

In [ ]:
df = df.drop('Extra')
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+------------------+---------------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|             sales|        Increament|     isBigOrder|Order_type|price_category|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+------------------+---------------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|15.299999999999999|21.299999999999997|Not a big order|     small| Not Expensive|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|             20.34|             26.34|Not a big order|     small| Not Expensive|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-0

In [ ]:
df = df.drop("Additional","isBigOrder")
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+------------------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|             sales|        Increament|Order_type|price_category|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+------------------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|15.299999999999999|21.299999999999997|     small| Not Expensive|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|             20.34|             26.34|     small| Not Expensive|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|              22.0|              

In [ ]:
df.select("Order_type").distinct().show()

+----------+
|Order_type|
+----------+
| Big Order|
|    Medium|
|     small|
+----------+

